In [3]:
import ktrain
from ktrain import text
import pandas as pd
from sklearn.model_selection import train_test_split

2021-08-08 10:32:19.757482: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-08 10:32:19.757530: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
train_df

In [4]:
train_df = pd.read_csv("gender-based-violence-tweet-classification/Train.csv")
random_seed = 12342
x_train, x_val, y_train, y_val = train_test_split(train_df['tweet'], 
train_df['type'], shuffle=True, test_size = 0.2, random_state=random_seed, stratify=train_df['type'])

In [2]:
y_val

NameError: name 'y_val' is not defined

In [6]:
import numpy

In [7]:
(x_train_bert, y_train_bert), (x_val_bert, y_val_bert), prepoc = text.texts_from_array(x_train=numpy.array(x_train), y_train=numpy.array(y_train), x_test= numpy.array(x_val), y_test=numpy.array(y_val),
                                          class_names=['sexual_violence', 'Physical_violence', 'emotional_violence',
       'Harmful_Traditional_practice', 'economic_violence'],
                                          preprocess_mode='bert',
                                          lang= 'en',
                                          maxlen=110,
                                          max_features= 35000)

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [8]:
model = text.text_classifier('bert', train_data=(x_train_bert, y_train_bert), preproc=prepoc)
learner = ktrain.get_learner(model, train_data=(x_train_bert, y_train_bert), val_data=(x_val_bert, y_val_bert), batch_size=16)

Is Multi-Label? False
maxlen is 110
2021-08-08 10:33:11.013000: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-08 10:33:11.013049: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-08 10:33:11.013086: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ws-3466f0d2-dd03-45cd-86a3-293ebb2ef476): /proc/driver/nvidia/version does not exist
2021-08-08 10:33:11.013432: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
done.


In [1]:
learner.autofit(1e-5)

NameError: name 'learner' is not defined

NameError: name 'learner' is not defined

1